# Tutorial: Numerical implementation of neutrino leakage scheme

## Author: Leo Werneck

# File-by-file implementation

# Step 1: `calc_leak.F90`



## Step 1.a: `ZelmaniLeak_CalcLeak`

## Step 1.b: `calc_leak`

## Step 1.c: `linterp2Dn`

This function performs [bilinear interpolation](https://en.wikipedia.org/wiki/Bilinear_interpolation) of a function $f$. Let $Q_{ij} = (x_{i},y_{j})$. The function returns,

$$
f_{\rm out}(x,y) = \frac{f_{\rm in}(Q_{11})(x_{2}-x)(y_{2}-y) + f_{\rm in}(Q_{21})(x-x_{1})(y_{2}-y) + f_{\rm in}(Q_{12})(x_{2}-x)(y-y_{1}) + f_{\rm in}(Q_{22})(x-x_{1})(y-y_{1})}{(x_{2}-x_{1})(y_{2}-y_{1})}.
$$

## Step 1.d: `linterp3Dn`

This function performs [trilinear interpolation](https://en.wikipedia.org/wiki/Trilinear_interpolation) of a function $f$. Let $Q_{ijk} = (x_{i},y_{j},z_{k})$. The function returns,

$$
\begin{split}
f_{\rm out}(x,y,z) = \left[(x_{2}-x_{1})(y_{2}-y_{1})(z_{2}-z_{1})\right]^{-1}
\big[&f_{\rm in}(Q_{111})(x_{2}-x)(y_{2}-y)(z_{2}-z)\\
+&f_{\rm in}(Q_{211})(x-x_{1})(y_{2}-y)(z_{2}-z)\\
+&f_{\rm in}(Q_{121})(x_{2}-x)(y-y_{1})(z_{2}-z)\\
+&f_{\rm in}(Q_{112})(x_{2}-x)(y_{2}-y)(z-z_{1})\\
+&f_{\rm in}(Q_{221})(x-x_{1})(y-y_{1})(z_{2}-z)\\
+&f_{\rm in}(Q_{212})(x-x_{1})(y_{2}-y)(z-z_{1})\\
+&f_{\rm in}(Q_{122})(x_{2}-x)(y-y_{1})(z-z_{1})\\
+&f_{\rm in}(Q_{222})(x-x_{1})(y-y_{1})(z-z_{1})
\big]
\end{split}.
$$

# Step 2: `calc_taus.F90`

This file contains 3 functions:

* `ZelmaniLeak_CalcTau()`
* `calc_taus()`
* `get_fermi_integral_leak()`

## Step 2.a: `get_fermi_integral_leak()`

Function `get_fermi_integral_leak()` contains hard-coded expressions for the Fermi-Dirac integrals

$$
F_{N}(\eta) = \int_{0}^{\infty}\frac{x^{N}dx}{e^{x-\eta}+1},
$$

for $N=0,\ldots,5$, following [Takahashi *et al.* (1978)](http://adsabs.harvard.edu/pdf/1978A%26A....67..185T). For $\eta>10^{-3}$, the function returns

$$
\begin{align}
F_{0}(\eta) &= \log_{10}\left(1+e^{\eta}\right),\\
F_{1}(\eta) &= \frac{\eta^{2}/2 + 1.6449}{1+\exp(-1.6855\eta)},\\
F_{2}(\eta) &= \frac{\eta^{3}/3 + 3.2899\eta}{1-\exp(-1.8246\eta)},\\
F_{3}(\eta) &= \frac{\eta^{4}/4 + 4.9348\eta^{2} + 11.3644}{1+\exp(-1.9039\eta)},\\
F_{4}(\eta) &= \frac{\eta^{5}/5 + 6.5797\eta^{3} + 45.4576\eta}{1-\exp(-1.9484\eta)},\\
F_{5}(\eta) &= \frac{\eta^{6}/6 + 8.2247\eta^{4} + 113.6439\eta^{2} + 236.5323}{1+exp(-1.9727\eta)},
\end{align}
$$

while for $\eta<10^{-3}$ it returns

$$
\begin{align}
F_{0}(\eta) &= \log_{10}\left(1+e^{\eta}\right),\\
F_{1}(\eta) &= e^{\eta}\left[1 + 0.2159\exp(0.8857\eta)\right]^{-1},\\
F_{2}(\eta) &= 2e^{\eta}\left[1 + 0.1095\exp(0.8908\eta)\right]^{-1},\\
F_{3}(\eta) &= 6e^{\eta}\left[1 + 0.0559\exp(0.9069\eta)\right]^{-1},\\
F_{4}(\eta) &= 24e^{\eta}\left[1 + 0.0287\exp(0.9257\eta)\right]^{-1},\\
F_{5}(\eta) &= 120e^{\eta}\left[1 + 0.0147\exp(0.9431\eta)\right]^{-1}.
\end{align}
$$